In [ ]:
import cv2
import numpy as np
import math
import pandas as pd
import os
import re

In [ ]:
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]

def read_image_data(folder_path):
    image_names = sorted(os.listdir(folder_path), key=natural_sort_key)
    if not image_names:
        print("No images found in the directory.")
        return []
    elif len(image_names) > 2:
        print(f"The names of the first three images in the directory are: {image_names[0]}, {image_names[1]}, {image_names[2]}")
    else:
        print("Not enough images to display three names.")
    image_paths = [os.path.join(folder_path, name) for name in image_names]
    return image_paths
class Hog_descriptor():
    def __init__(self, cell_size=16, bin_size=9):
        self.cell_size = cell_size
        self.bin_size = bin_size
        self.angle_unit = 360 / self.bin_size

    def extract(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise FileNotFoundError(f"The image at {img_path} could not be loaded.")
        if np.max(img) == 0:
            raise ValueError("The maximum value in the image is 0, which can lead to division by zero.")
        img = np.sqrt(img / float(np.max(img)))
        img = img * 255

        height, width = img.shape
        gradient_magnitude, gradient_angle = self.global_gradient(img)
        cell_gradient_vector = np.zeros((int(height / self.cell_size), int(width / self.cell_size), self.bin_size))
        for i in range(cell_gradient_vector.shape[0]):
            for j in range(cell_gradient_vector.shape[1]):
                cell_magnitude = gradient_magnitude[i * self.cell_size:(i + 1) * self.cell_size,
                                 j * self.cell_size:(j + 1) * self.cell_size]
                cell_angle = gradient_angle[i * self.cell_size:(i + 1) * self.cell_size,
                             j * self.cell_size:(j + 1) * self.cell_size]
                cell_gradient_vector[i][j] = self.cell_gradient(cell_magnitude, cell_angle)

        hog_vector = []
        for i in range(cell_gradient_vector.shape[0]):
            for j in range(cell_gradient_vector.shape[1]):
                block_vector = cell_gradient_vector[i][j].tolist()
                hog_vector.extend(block_vector)
        return hog_vector[:256]  # Adjust the slice to control feature length

    def global_gradient(self, img):
        gradient_values_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=5)
        gradient_values_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=5)
        gradient_magnitude = cv2.addWeighted(gradient_values_x, 0.5, gradient_values_y, 0.5, 0)
        gradient_angle = cv2.phase(gradient_values_x, gradient_values_y, angleInDegrees=True)
        return gradient_magnitude, gradient_angle

    def cell_gradient(self, cell_magnitude, cell_angle):
        orientation_centers = [0] * self.bin_size
        for i in range(cell_magnitude.shape[0]):
            for j in range(cell_magnitude.shape[1]):
                gradient_strength = cell_magnitude[i][j]
                gradient_angle = cell_angle[i][j]
                min_angle, max_angle, mod = self.get_closest_bins(gradient_angle)
                orientation_centers[min_angle] += (gradient_strength * (1 - (mod / self.angle_unit)))
                orientation_centers[max_angle] += (gradient_strength * (mod / self.angle_unit))
        return orientation_centers

    def get_closest_bins(self, gradient_angle):
        idx = int(gradient_angle / self.angle_unit)
        mod = gradient_angle % self.angle_unit
        if idx == self.bin_size:
            return idx - 1, (idx) % self.bin_size, mod
        return idx, (idx + 1) % self.bin_size, mod

def process_folder(folder_path):
    hog = Hog_descriptor()
    features_list = []
    filenames = sorted(os.listdir(folder_path))  
    for filename in filenames:
        if filename.endswith('.png'):
            img_path = os.path.join(folder_path, filename)
            try:
                features = hog.extract(img_path)
                features_list.append([filename] + features) 
            except Exception as e:
                print(f"Error processing {filename}: {e}")
    df = pd.DataFrame(features_list, columns=['Name'] + [f'HOG_{i}' for i in range(256)]) 
    df.to_csv('negative.csv', index=False)
    print("All features saved to hog_features.csv")

folder_path = r'D:\ICIIT2024\Dataset Rice seed\Thien_uu\Thien_uu\Negative'
process_folder(folder_path)


In [ ]:

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]
def read_image_data(folder_path):
    image_names = sorted(os.listdir(folder_path), key=natural_sort_key)
    if not image_names:
        print("No images found in the directory.")
        return []
    elif len(image_names) > 2:
        print(f"The names of the first three images in the directory are: {image_names[0]}, {image_names[1]}, {image_names[2]}")
    else:
        print("Not enough images to display three names.")
    image_paths = [os.path.join(folder_path, name) for name in image_names]
    return image_paths

class Hog_descriptor():
    def __init__(self, cell_size=16, bin_size=9):
        self.cell_size = cell_size
        self.bin_size = bin_size
        self.angle_unit = 360 / self.bin_size

    def extract(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise FileNotFoundError(f"The i3mage at {img_path} could not be loaded.")
        if np.max(img) == 0:
            raise ValueError("The maximum value in the image is 0, which can lead to division by zero.")
        img = np.sqrt(img / float(np.max(img)))
        img = img * 255

        height, width = img.shape
        gradient_magnitude, gradient_angle = self.global_gradient(img)
        cell_gradient_vector = np.zeros((int(height / self.cell_size), int(width / self.cell_size), self.bin_size))
        for i in range(cell_gradient_vector.shape[0]):
            for j in range(cell_gradient_vector.shape[1]):
                cell_magnitude = gradient_magnitude[i * self.cell_size:(i + 1) * self.cell_size,
                                 j * self.cell_size:(j + 1) * self.cell_size]
                cell_angle = gradient_angle[i * self.cell_size:(i + 1) * self.cell_size,
                             j * self.cell_size:(j + 1) * self.cell_size]
                cell_gradient_vector[i][j] = self.cell_gradient(cell_magnitude, cell_angle)

        hog_vector = []
        for i in range(cell_gradient_vector.shape[0]):
            for j in range(cell_gradient_vector.shape[1]):
                block_vector = cell_gradient_vector[i][j].tolist()
                hog_vector.extend(block_vector)
        return hog_vector[:256]  # Adjust the slice to control feature length

    def global_gradient(self, img):
        gradient_values_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=5)
        gradient_values_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=5)
        gradient_magnitude = cv2.addWeighted(gradient_values_x, 0.5, gradient_values_y, 0.5, 0)
        gradient_angle = cv2.phase(gradient_values_x, gradient_values_y, angleInDegrees=True)
        return gradient_magnitude, gradient_angle

    def cell_gradient(self, cell_magnitude, cell_angle):
        orientation_centers = [0] * self.bin_size
        for i in range(cell_magnitude.shape[0]):
            for j in range(cell_magnitude.shape[1]):
                gradient_strength = cell_magnitude[i][j]
                gradient_angle = cell_angle[i][j]
                min_angle, max_angle, mod = self.get_closest_bins(gradient_angle)
                orientation_centers[min_angle] += (gradient_strength * (1 - (mod / self.angle_unit)))
                orientation_centers[max_angle] += (gradient_strength * (mod / self.angle_unit))
        return orientation_centers

    def get_closest_bins(self, gradient_angle):
        idx = int(gradient_angle / self.angle_unit)
        mod = gradient_angle % self.angle_unit
        if idx == self.bin_size:
            return idx - 1, (idx) % self.bin_size, mod
        return idx, (idx + 1) % self.bin_size, mod

def process_folder(folder_path):
    hog = Hog_descriptor()
    features_list = []

    image_names = sorted(os.listdir(folder_path), key=natural_sort_key)
    if not image_names:
        print("No images found in the directory.")
        return
    elif len(image_names) > 2:
        print(f"The names of the first three images in the directory are: {image_names[0]}, {image_names[1]}, {image_names[2]}")
    else:
        print("Not enough images to display three names.")

    for image_name in image_names:
        image_path = os.path.join(folder_path, image_name)
        hog_features = hog.extract(image_path)
        features_list.append([os.path.basename(image_path)] + hog_features)

    df = pd.DataFrame(features_list, columns=['Name'] + [f'HOG_{i}' for i in range(256)]) 
    df.to_csv('positive_text.csv', index=False)
    #print("All features saved to hog_features_cell16_bin9_256ft_3.csv")

folder_path = r'D:\ICIIT2024\Dataset Rice seed\BC-15\BC-15\BC-15'
process_folder(folder_path)

In [ ]:
df1 = pd.read_csv("positive.csv")
df1['class'] = 'positive' 
df2 = pd.read_csv("negative.csv")
df2['class'] = 'negative'
result = pd.concat([df1, df2], ignore_index=True)
result.to_csv("Xi_HOG.csv")

In [ ]:
df_hog = pd.read_csv("Xi_HOG.csv")
df_hog_drop = df_hog.iloc[:, 2:]
df_hog_drop.shape

In [ ]:
df_combine = pd.read_csv(r"D:\ICIIT2024\Basic_GLCM_LBP_GIST\Xi23++.csv")
df_combine.head()
# df_combine_drop = df_combine.iloc[:, 1:]
df_combine_drop = df_combine.drop(columns=['class'])
df_combine_drop.shape

In [ ]:
df_combined = pd.concat([df_combine_drop, df_hog_drop], axis=1)

df_combined.to_csv('Gist_GLCM_LBP_Basic_Xi23.csv', index=False)
df_combined.shape